In [1]:
import numpy as np

# Layers

### Linear layer 

In [ ]:
class Linear:
    def __init__(self):
        pass

    def forward(self):
        pass

    def backward(self):
        pass

    def step(self):
        pass

### Sigmoid layer

In [ ]:
class Sigmoid:
    def __init__(self):
        pass

    def forward(self):
        pass

    def backward(self):
        pass

### RELU layer

In [ ]:
class ReLU:
    def __init__(self):
        pass

    def forward(self):
        pass

    def backward(self):
        pass

### Softmax Layer

In [ ]:
class ReLU:
    def __init__(self):
        pass

    def forward(self):
        pass

    def backward(self):
        pass

# Loss classes

### Mean Squared Error

In [ ]:
class MSE:
    def __init__(self):
        pass

    def forward(self):
        pass

    def backward(self):
        pass

### Cross entropy

In [ ]:
class CrossEntropy:
    def __init__(self):
        pass

    def forward(self):
        pass

    def backward(self):
        pass

# Multi Layer Perceptron

In [ ]:
class MLP:
    def __init__(self):
        pass

    def forward(self):
        pass

    def backward(self):
        pass

    def update_weigths(self):
        pass

    def train(self):
        pass

    def predict(self):
        pass